# What do we want?

1. To create a CSV of questions
    - Each question will have a subject
    - There will be a folder called `images` to hold any pictures related to the question
2. We want to be able to keep track of some data points about each quiz
3. We want to have an email sent to us each day with a link to a page
    - On the page, we'll answer the questions
    - Once done, the quiz results will be saved and persisted

# Step 1: Creating questions

- This will be easy
    - Will do it in Excel and save to folder

In [1]:
from datetime import datetime
import pandas as pd
import time

In [2]:
df_test_questions = pd.read_csv('test_quiz.csv')

# Step 2: Writing function to read in questions from CSV and create question widgets

- First, we define our `create_multipleChoice_widget` function

In [3]:
from ipywidgets import widgets, Layout, Box
from IPython.display import display, clear_output

In [4]:
def create_multipleChoice_widget(category, description, options, correct_answer, img_filepath = None):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    options_w_letters = [f'{alphabet[i]}) {option}' for i, option in enumerate(options)]
    
    radio_options = [(words, i) for i, words in enumerate(options_w_letters)]
    
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    
    description_out = widgets.Output(layout=Layout(width='auto'))
    
    if img_filepath:
        image = open(img_filepath, 'rb').read()
        image_out = widgets.Image(value=image, 
                                  format='png', 
                                  width='200px', 
                                  height='auto',
                                 )
    
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        flag = a==correct_answer_index
        if flag:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            correct_answer_letter = alphabet[correct_answer_index]
            s = '\x1b[5;30;41m' + f"incorrect, the correct answer was {correct_answer_letter}" + '\x1b[0m' +"\n"
        with feedback_out:
            global n_questions_answered
            n_questions_answered += 1
            print(s)
            s_results = pd.Series({'Category':category, 
                                   'Question':description, 
                                   'Correct Answer':correct_answer, 
                                   'Submitted Answer':options[a], 
                                   'Score':int(flag)})
            global quiz_results
            quiz_results.append(s_results)
            alternativ.close()
            check.layout.display = 'none'
        return
    
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    
    check_out = widgets.HBox([check])
    
    if img_filepath:
            return widgets.VBox([description_out, 
                         image_out,
                         alternativ, 
                         widgets.HBox([check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 
    
    return widgets.VBox([description_out, 
                         alternativ, 
                         check_out, 
                         feedback_out], 
                        layout=Layout(
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            width='auto')) 

- Next, we define a function whose parameter is the CSV of questions

In [5]:
def create_questions(df_questions):
    list_widgets = []
    for question_idx, s_question in df_questions.iterrows():
        category = s_question.loc['Category']
        description = s_question.loc['Question']
        options = s_question.loc['Options'].split(', ')
        correct_answer = s_question.loc['Correct Answer']
        image_filename = s_question.loc['Image Filename']
        
        if '.png' in str(image_filename):
            image_filepath = f'images/{image_filename}'
        else:
            image_filepath = None
        
        question_widget = create_multipleChoice_widget(category, description, options, correct_answer, image_filepath)
        list_widgets.append(question_widget)
    return list_widgets

In [6]:
list_widgets = create_questions(df_test_questions)

# Step 3: Writing function to create quiz

In [7]:
def create_quiz(list_widgets):
    quiz_timestamp = datetime.now().strftime('%Y%m%d%M%S')
    n_questions = len(list_widgets)
    global n_questions_answered
    n_questions_answered = 0    
    global quiz_results
    quiz_results = []
    
    def on_button_clicked(b):
        if n_questions_answered == n_questions:
            df_final_results = pd.DataFrame(quiz_results)
            df_final_results['quiz'] = quiz_timestamp
            time.sleep(2)
            clear_output()
            final_score = df_final_results['Score'].sum()
            final_score_pct = round(100 * (final_score / float(n_questions)), 2)
            final_score_message = f'Final Score: {final_score}/{n_questions} ({final_score_pct}%) \n'
            print(final_score_message)
            print(df_final_results)
    
    for widget in list_widgets:
        display(widget)
        for child in widget.children:
            if type(child)==widgets.widget_box.HBox:
                child.children[0].on_click(on_button_clicked)

In [8]:
create_quiz(list_widgets)

Final Score: 1/3 (33.33%) 

    Category                         Question Correct Answer Submitted Answer  \
0       Math                   What is 1 + 1?              2                5   
1       Math                     What is 2+2?              4                1   
2  Geography  What is the capital of Ontario?        Toronto          Toronto   

   Score          quiz  
0      0  202007294850  
1      0  202007294850  
2      1  202007294850  


# Follow ups

- Seems like everything is working
    - Now we just need to:
        1. Determine ow the questions are selected to create the quiz
            - I assume I'll make it be random
        2. Figure out how to use `voila` to turn this into a dashboard
        3. Create an actual list of questions
        4. Write the code to persist `df_final_results` into a massive dataframe
        5. Figure out how to automatically send the email